# Importaciones

In [ ]:
import pandas as pd
import os
from textblob import TextBlob

Creación de directorios si no existen

In [ ]:
os.mkdir("Datawarehouse") if not os.path.exists("Datawarehouse") else None

# Carga de dataframes

Se realiza una carga de los dataframes exportados previamente, para acelerar el proceso de testeo, ya que los pasos anteriores requieren de un alto consumo de tiempo.

In [ ]:
df_restaurantes = pd.read_parquet(r'Datalake\restaurantes.parquet')
df_reviews = pd.read_parquet(r'Datalake\reviews.parquet')
df_estados = pd.read_parquet(r'Datalake\estados.parquet')

## Eliminación de datos duplicados según ID y restaurantes sin id_estado

In [ ]:
df_restaurantes = df_restaurantes.drop_duplicates(subset=["id_restaurante"], keep='last')
df_reviews = df_reviews.drop_duplicates(subset=["id_resena"], keep='last')
df_restaurantes = df_restaurantes.dropna(subset=['id_estado'])

# Conversión de datos por columna

In [ ]:
df_restaurantes.loc[:, 'cantidad_resenas'] = df_restaurantes['cantidad_resenas'].astype('uint32')
df_restaurantes.loc[:, 'calificacion'] = df_restaurantes['calificacion'].astype('float16')
df_restaurantes.loc[:, 'id_estado'] = df_restaurantes['id_estado'].astype('uint8')
df_reviews.loc[:, 'calificacion'] = df_reviews['calificacion'].astype('float16')
df_reviews.loc[:, 'anio'] = df_reviews['anio'].astype('float16')
df_reviews.loc[:, 'puntaje_de_sentimiento'] = df_reviews['puntaje_de_sentimiento'].astype('float16')
df_estados.loc[:, 'id_estado'] = df_estados['id_estado'].astype('uint8')
df_estados.loc[:, 'Population'] = df_estados['Population'].astype('uint32')

# Ejecución de análisis de sentimiento

Los elementos sin procesar tienen un puntaje de sentimiento = 100, los que serán almacenados temporalmente en df_analisis para ser procesados y luego se reemplazarán los valores correspondientes en df_reviews

In [ ]:
#extracción de líneas a analizar
df_analisis = df_reviews[df_reviews['puntaje_de_sentimiento'] == 100].copy()

## Análisis de sentimiento

In [ ]:
def assign_sentiment_score(text):
    # Calcular el sentimiento usando TextBlob
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    
    # Mapear el sentimiento a una escala de 0 a 10
    # -1 <= sentiment_score <= 1
    # 0 <= mapped_score <= 10
    mapped_score = (sentiment_score + 1) * 5
    
    # Redondear el valor del sentimiento
    rounded_score = round(mapped_score, 1)
    
    return rounded_score

# Aplicar la función a la columna 'texto' y crear una nueva columna 'puntaje_de_sentimiento'
df_analisis['puntaje_de_sentimiento'] = df_analisis['texto'].apply(assign_sentiment_score)

# Verificar los resultados
print(df_analisis[['texto', 'puntaje_de_sentimiento']].head())

## Actualización de datos post-análisis

In [ ]:
df_reviews.loc[df_reviews['id_resena'].isin(df_analisis['id_resena']), 'puntaje_de_sentimiento'] = df_analisis['puntaje_de_sentimiento']

In [ ]:
df_reviews.head()

In [ ]:
# Debug
df_reviews['puntaje_de_sentimiento'] = 100

In [ ]:
df_reviews.head()

In [ ]:
df_restaurantes.to_parquet(r"Datawarehouse\restaurantes.parquet")
df_reviews.to_parquet(r"Datawarehouse\reviews.parquet")
df_estados.to_parquet(r"Datawarehouse\estados.parquet")